In [1]:
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Brain-Tumor-Classification'

In [2]:
def load_data():
    labels = {}
    labels_r = {}
    idx = -1
    data = []
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
        for file in os.listdir(f'./data/{folder}/'):
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(112,112))/255.0
            data.append([img,idx])
    np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.0625)
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [3]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

100%|█████████████████████████████████████████████| 4/4 [00:05<00:00,  1.41s/it]


In [4]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device)
y_test = torch.from_numpy(np.array(y_test)).to(device)

In [5]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [6]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [7]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.activation = ReLU()
        self.conv1 = Conv2d(3,5,(3,3))
        self.conv1batchnorm = BatchNorm2d(5)
        self.conv2 = Conv2d(5,10,(3,3))
        self.conv2batchnorm = BatchNorm2d(10)
        self.conv3 = Conv2d(10,15,(3,3))
        self.conv3batchnorm = BatchNorm2d(15)
        self.conv4 = Conv2d(15,20,(3,3))
        self.conv4batchnorm = BatchNorm2d(20)
        self.linear1 = Linear(20*5*5,512)
        self.linear1batchnorm = BatchNorm1d(512)
        self.linear2 = Linear(512,1024)
        self.linear2batchnorm = BatchNorm1d(1024)
        self.linear3 = Linear(1024,512)
        self.linear3batchnorm = BatchNorm1d(512)
        self.output = Linear(512,len(labels))
    
    def forward(self,X):
        preds = self.max_pool2d(self.activation(self.conv1batchnorm(self.conv1(X))))
        preds = self.max_pool2d(self.activation(self.conv2batchnorm(self.conv2(preds))))
        preds = self.max_pool2d(self.activation(self.conv3batchnorm(self.conv3(preds))))
        preds = self.max_pool2d(self.activation(self.conv4batchnorm(self.conv4(preds))))
#         print(preds.shape)
        preds = preds.view(-1,20*5*5)
        preds = self.activation(self.linear1batchnorm(self.linear1(preds)))
        preds = self.activation(self.linear2batchnorm(self.linear2(preds)))
        preds = self.activation(self.linear3batchnorm(self.linear3(preds)))
        preds = self.output(preds)
        return preds

In [8]:
model = Model().to(device)

In [9]:
criterion = CrossEntropyLoss()

In [10]:
from torch.optim import *

In [11]:
optimizer = Adam(model.parameters(),lr=0.001)

In [20]:
epochs = 1000

In [21]:
batch_size = 8

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].view(-1,3,112,112).to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion))/2})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

 40%|████████████████▏                       | 405/1000 [17:14<25:34,  2.58s/it]